<a href="https://colab.research.google.com/github/boheling/healthAI/blob/main/SFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install necessary packages
!pip install transformers datasets trl --quiet
#!huggingface-cli login

from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from datasets import load_dataset
from trl import SFTTrainer

# Set wandb logging directory to a folder in Google Drive
import os
os.environ["WANDB_DIR"] = "/content/drive/MyDrive/SFT/wandb_logs"

# Optionally, initialize wandb explicitly if needed:
import wandb
wandb.init(project="my_sft_project", dir=os.environ["WANDB_DIR"])

# Use the distill model
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Save the pre-trained model for backup
model.save_pretrained("/content/drive/MyDrive/SFT/pretrain_model")
tokenizer.save_pretrained("/content/drive/MyDrive/SFT/pretrain_model")

# Q&A healthcare doctor dataset
dataset = load_dataset("lavita/ChatDoctor-HealthCareMagic-100k", split="train[:10%]")

def tokenize_function(examples):
    texts = []
    for i in range(len(examples["instruction"])):
        # Build the text with structure
        text = f"Instruction: {examples['instruction'][i]}\n"
        # Process 'input'
        if examples["input"][i]:
            if isinstance(examples["input"][i], list):
                input_text = " ".join(examples["input"][i])
            else:
                input_text = examples["input"][i]
            text += f"Input: {input_text}\n"
        # Process 'output'
        if examples["output"][i]:
            if isinstance(examples["output"][i], list):
                output_text = " ".join(examples["output"][i])
            else:
                output_text = examples["output"][i]
            text += f"Output: {output_text}\n"
        texts.append(text)
    return tokenizer(texts, truncation=True, max_length=128)

# Tokenize the dataset and remove original text columns
unfiltered_tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["instruction", "input", "output"]
)

# Filter out examples with empty tokenization results
tokenized_dataset = unfiltered_tokenized_dataset.filter(lambda x: len(x["input_ids"]) > 0)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
print(unfiltered_tokenized_dataset[0])

{'input_ids': [151646, 16664, 25, 1416, 498, 525, 264, 10668, 11, 4486, 4226, 279, 6457, 4755, 3118, 389, 279, 8720, 594, 4008, 624, 2505, 25, 358, 38726, 705, 419, 6556, 8266, 279, 4361, 3054, 374, 37860, 979, 600, 572, 11699, 1495, 13, 358, 3937, 311, 279, 14852, 11435, 650, 42180, 1541, 11, 438, 600, 6679, 311, 5244, 600, 2666, 48294, 782, 13, 358, 1430, 311, 21982, 275, 714, 432, 39364, 2525, 700, 496, 4636, 4633, 7215, 86769, 323, 6084, 369, 2421, 4115, 11, 600, 2058, 2666, 279, 1852, 496, 3216, 279, 1616, 11, 421, 600, 10962, 1495, 476, 2444, 1495, 11, 847, 1968, 653, 537, 12616, 11, 1172, 979, 600, 1366, 311, 3271, 2163, 1221, 600, 2666, 279, 4361, 1879, 374, 37860, 496, 1597, 432, 374, 4622, 22350, 43676, 518], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [4]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/SFT/sft_output",  # save checkpoints here
    logging_dir="/content/drive/MyDrive/SFT/sft_logs",
    per_device_train_batch_size=2,   # Adjust as necessary
    num_train_epochs=2,              # Increase for more training
    logging_steps=10,
    save_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    fp16=True,                       # Mixed precision for T4 GPU
    dataloader_num_workers=2,        # Adjust based on your CPU
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
)

# Evaluate the pre-trained model (baseline)
print("Evaluating pre-trained model...")
pretrain_metrics = trainer.evaluate()
print("Pre-training evaluation metrics:", pretrain_metrics)

# Sample prompt before training
prompt = "Q: What could be wrong with lower back pain in a cancer patient?\nA:"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=100, do_sample=True, temperature=0.7, top_p=0.9)
print("Pre-training output:", tokenizer.decode(outputs[0], skip_special_tokens=True))

# Start fine-tuning
print("Starting fine-tuning...")
trainer.train()

# Save the fine-tuned (post-train) model
model.save_pretrained("/content/drive/MyDrive/SFT/posttrain_model")
tokenizer.save_pretrained("/content/drive/MyDrive/SFT/posttrain_model")


# Evaluate the fine-tuned model
print("Evaluating fine-tuned model...")
posttrain_metrics = trainer.evaluate()
print("Post-training evaluation metrics:", posttrain_metrics)

# Sample prompt after training
prompt = "Q: What could be wrong with lower back pain in a cancer patient?\nA:"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=100, do_sample=True, temperature=0.7, top_p=0.9)
print("Post-training output:", tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Converting train dataset to ChatML:   0%|          | 0/11216 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/11216 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/11216 [00:00<?, ? examples/s]

Evaluating pre-trained model...


...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: boheling (boheling-stanford-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Pre-training evaluation metrics: {'eval_loss': 4.312994003295898, 'eval_model_preparation_time': 0.005, 'eval_runtime': 79.443, 'eval_samples_per_second': 141.183, 'eval_steps_per_second': 17.648}
Pre-training output: Q: What could be wrong with lower back pain in a cancer patient?
A: The lower back pain in a cancer patient is likely to be due to a number of factors, including:

1. **Cancer itself** - Cancer can have a significant impact on the lower back, often leading to issues such as pain, stiffness, or difficulty moving the legs.

2. **Infiltration of cancer cells into the lower back muscles** - This can cause localized pain, stiffness, or difficulty
Starting fine-tuning...


Step,Training Loss,Validation Loss,Model Preparation Time
50,2.928900,3.092282,0.005000
100,3.302500,3.034015,0.005000
150,2.860400,3.003200,0.005000


KeyboardInterrupt: 